# Books Recommendation Data Preperation for Deconfounder


## Setup

In [ ]:
!pip install tensorflow_probability

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import numpy.random as npr
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import statsmodels.api as sm

from tensorflow_probability import edward2 as ed
from sklearn.datasets import load_breast_cancer
from pandas.plotting import scatter_matrix
from scipy import sparse, stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, mean_squared_error, r2_score
from sklearn.utils import shuffle

import sys
import os


import matplotlib
matplotlib.rcParams.update({'font.sans-serif' : 'Helvetica',
                            'axes.labelsize': 10,
                            'xtick.labelsize' : 6,
                            'ytick.labelsize' : 6,
                            'axes.titlesize' : 10})
import matplotlib.pyplot as plt

import seaborn as sns
color_names = ["windows blue",
               "amber",
               "crimson",
               "faded green",
               "dusty purple",
               "greyish"]
colors = sns.xkcd_palette(color_names)
sns.set(style="white", palette=sns.xkcd_palette(color_names), color_codes = False)

TensorFlow 1.x selected.


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install weave

  Using cached weave-0.17.0.tar.gz (573 kB)
  Using cached weave-0.16.0.tar.gz (573 kB)
  Using cached weave-0.15.0.tar.gz (589 kB)
ERROR: Could not find a version that satisfies the requirement weave (from versions: 0.15.0, 0.16.0, 0.17.0)
ERROR: No matching distribution found for weave


In [ ]:
OUT_DATA_DIR = '/content/'
OUT_DIR = '../res/factorfit/'

In [ ]:
books = pd.read_csv('/content/books_clean.csv')
users = pd.read_csv('/content/users_clean.csv')
ratings = pd.read_csv('/content/ratings_clean.csv')

In [ ]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",34
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",34
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",34


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
#rename User-ID
ratings.rename({"User-ID": "user_id", "Book-Rating":"rating"}, axis=1, inplace=True)
users.rename({"User-ID": "user_id"}, axis=1, inplace=True)

## Data Preperation- Mappings for UserID and ISBN

Find user and Item popularity, convert user id and ISBN to numeric id. Store Mappings

In [ ]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [ ]:
user_activity = get_count(ratings, 'user_id')
item_popularity = get_count(ratings, 'ISBN')


In [ ]:
user_activity

,user_id,size
0,2,1
1,8,17
2,9,3
3,10,1
4,12,1
...,...,...
92101,278846,1
92102,278849,4
92103,278851,23
92104,278852,1


In [ ]:
item_popularity

,ISBN,size
0,0000913154,1
1,0001010565,2
2,0001046438,1
3,0001046713,1
4,000104687X,1
...,...,...
270146,B000234N76,1
270147,B000234NC6,1
270148,B00029DGGO,1
270149,B0002JV9PY,1


In [ ]:
def get_unique_ids():

  unique_uid = user_activity.user_id
  unique_bid = item_popularity.ISBN

  n_users = len(unique_uid)
  n_items = len(unique_bid)

  with open(os.path.join(OUT_DATA_DIR, 'unique_uid.txt'), 'w') as f:
    for uid in unique_uid:
        f.write('%s\n' % uid)

  with open(os.path.join(OUT_DATA_DIR, 'unique_bid.txt'), 'w') as f:
    for sid in unique_bid:
        f.write('%s\n' % sid)

  print(n_users, n_items)
  return unique_uid, unique_bid, n_users, n_items

In [ ]:
unique_uid, unique_bid, n_users, n_items = get_unique_ids()

92106 270151


In [ ]:
book2id = dict((sid, i) for (i, sid) in enumerate(unique_bid))
user2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))
id2user = dict((i, uid) for (i, uid) in enumerate(unique_uid))
id2book = dict((i, sid) for (i, sid) in enumerate(unique_bid))


In [ ]:
def numerize(tp):
    uid = list(map(lambda x: user2id[x], tp['user_id']))
    sid = list(map(lambda x: book2id[x], tp['ISBN']))
    tp.loc[:, 'uid'] = uid
    tp.loc[:, 'bid'] = sid
    return tp[['uid', 'bid', 'rating']]

In [ ]:
df = numerize(ratings)


In [ ]:
df

,uid,bid,rating
0,91362,45921,0
1,91363,22731,5
2,91364,92659,0
3,91365,111478,3
4,91365,111512,6
...,...,...,...
1031131,91358,197689,0
1031132,91358,224480,9
1031133,91359,143500,0
1031134,91360,108009,10


In [ ]:
ratings

,user_id,ISBN,rating,uid,bid
0,276725,034545104X,0,91362,45921
1,276726,0155061224,5,91363,22731
2,276727,0446520802,0,91364,92659
3,276729,052165615X,3,91365,111478
4,276729,0521795028,6,91365,111512
...,...,...,...,...,...
1031131,276704,0876044011,0,91358,197689
1031132,276704,1563526298,9,91358,224480
1031133,276706,0679447156,0,91359,143500
1031134,276709,0515107662,10,91360,108009


## Prepare Data for Confounder

In [ ]:
def split_train_test_proportion(data, uid, test_prop=0.5, random_seed=0):
    data_grouped_by_user = data.groupby(uid)
    tr_list, te_list = list(), list()

    np.random.seed(random_seed)

    for u, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if u % 5000 == 0:
            print("%d users sampled" % u)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [ ]:
train_data, vad_data = split_train_test_proportion(df, 'uid', test_prop=0.2, random_seed=12345)

0 users sampled
5000 users sampled
10000 users sampled
15000 users sampled
20000 users sampled
25000 users sampled
30000 users sampled
35000 users sampled
40000 users sampled
45000 users sampled
50000 users sampled
55000 users sampled
60000 users sampled
65000 users sampled
70000 users sampled
75000 users sampled
80000 users sampled
85000 users sampled
90000 users sampled


In [ ]:
print("There are total of %d unique users in the training set and %d unique users in the entire dataset" % \
(len(pd.unique(train_data['uid'])), len(unique_uid)))

There are total of 92106 unique users in the training set and 92106 unique users in the entire dataset


In [ ]:
def move_to_fill(part_data_1, part_data_2, unique_id, key):
    # move the data from part_data_2 to part_data_1 so that part_data_1 has the same number of unique "key" as unique_id
    part_id = set(pd.unique(part_data_1[key]))
    
    left_id = list()
    for i, _id in enumerate(unique_id):
        if _id not in part_id:
            left_id.append(_id)
            
    move_idx = part_data_2[key].isin(left_id)
    part_data_1 = part_data_1.append(part_data_2[move_idx])
    part_data_2 = part_data_2[~move_idx]
    return part_data_1, part_data_2

In [ ]:
train_data, vad_data = move_to_fill(train_data, vad_data, np.arange(n_items), 'bid')

In [ ]:
train_data.to_csv(os.path.join(OUT_DATA_DIR, 'train.csv'), index=False)
vad_data.to_csv(os.path.join(OUT_DATA_DIR, 'validation.csv'), index=False)
df.to_csv(os.path.join(OUT_DATA_DIR, 'train_full.csv'), index=False)

In [ ]:
unique_uid = list()
with open(os.path.join(OUT_DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())

unique_sid = list()
with open(os.path.join(OUT_DATA_DIR, 'unique_bid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_users = len(unique_uid)
n_items = len(unique_sid)

In [ ]:
def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    rows, cols, vals = np.array(tp['uid']), np.array(tp['bid']), np.array(tp['rating']) 
    data = sparse.csr_matrix((vals, (rows, cols)), dtype=np.float32, shape=shape)
    return data


train_data = load_data(os.path.join(OUT_DATA_DIR, 'train.csv'))
vad_data = load_data(os.path.join(OUT_DATA_DIR, 'validation.csv'))